Elisabeth Gangwer 

Assignment 3

<span style="font-size: 12.000000pt; font-family: 'TimesNewRomanPS'; font-weight: 700">Problem 1</span>

<span style="font-size: 12.000000pt; font-family: 'TimesNewRomanPSMT'">In the last homework assignment, you created a list of territories that required training because they did not have at least one month during 2013 with sales of more than $750,000. Working with this sample principle, use a CASE statement to create a list of all territories in alphabetical order, with a new column called "TrainingRequired" that has a value of either "Yes" or "No."&nbsp;</span>

In [10]:
SELECT DISTINCT t.Name AS Territory
    , CASE WHEN MAX(sum(soh.SubTotal)) OVER(PARTITION BY t.Name) > 750000
        THEN 'No'
        ELSE 'Yes'
    END AS TrainingRequired 
FROM sales.SalesOrderHeader soh
INNER JOIN sales.SalesTerritory t
ON soh.TerritoryID = t.TerritoryID
WHERE DATEPART(YEAR,soh.OrderDate) = 2013
GROUP BY t.Name
    , DATEPART(MONTH, soh.OrderDate)
    , DATEPART(YEAR,soh.OrderDate)
ORDER BY Name 

(10 rows affected)

Total execution time: 00:00:00.050

Territory,TrainingRequired
Australia,Yes
Canada,No
Central,Yes
France,No
Germany,Yes
Northeast,Yes
Northwest,No
Southeast,Yes
Southwest,No
United Kingdom,Yes


<span style="font-size: 12.000000pt; font-family: 'TimesNewRomanPS'; font-weight: 700">Problem 2</span>

<span style="font-size: 12.000000pt; font-family: 'TimesNewRomanPSMT'">The sales manager wants to know how many sales orders were shipped to each country and state/province. Your result set should contain three columns: Country, StateProvince, and TotalOrders in alphabetical order by country, then showing the greatest number of TotalOrders.</span>

<span style="font-size: 12.000000pt; font-family: 'TimesNewRomanPSMT'">Part A<br>Write a single query that shows the total number of sales orders from each country and each state/province within each country. Use the full name of each country and state/province.&nbsp;</span>

In [11]:
SELECT cr.Name AS Country
    , sp.Name AS StateProvince
    , COUNT(soh.SalesOrderID) AS TotalOrders   
FROM sales.SalesOrderHeader soh 
INNER JOIN person.Address a
    ON soh.ShipToAddressID = a.AddressID
INNER JOIN Person.StateProvince sp
    ON a.StateProvinceID = sp.StateProvinceID
INNER JOIN person.CountryRegion cr
    ON sp.CountryRegionCode = cr.CountryRegionCode
GROUP BY ROLLUP (cr.Name, sp.Name)
ORDER BY cr.Name ASC, TotalOrders DESC

(77 rows affected)

Total execution time: 00:00:00.089

Country,StateProvince,TotalOrders
NULL,NULL,31465
Australia,NULL,6843
Australia,New South Wales,3009
Australia,Victoria,1710
Australia,Queensland,1461
Australia,South Australia,466
Australia,Tasmania,197
Canada,NULL,4067
Canada,British Columbia,3472
Canada,Ontario,339


<span style="font-size: 12.000000pt; font-family: 'TimesNewRomanPSMT'">Part B<br>Modify your query to only show cases with 100 or more orders, sorted in the same fashion as Part A.&nbsp;</span>

In [12]:
SELECT cr.Name AS Country
    , sp.Name AS StateProvince
    , COUNT(soh.SalesOrderID) AS TotalOrders   
FROM sales.SalesOrderHeader soh 
INNER JOIN person.Address a
    ON soh.ShipToAddressID = a.AddressID
INNER JOIN Person.StateProvince sp
    ON a.StateProvinceID = sp.StateProvinceID
INNER JOIN person.CountryRegion cr
    ON sp.CountryRegionCode = cr.CountryRegionCode
GROUP BY ROLLUP (cr.Name, sp.Name)
HAVING COUNT(soh.SalesOrderID) > 99
ORDER BY cr.Name ASC, TotalOrders DESC

(33 rows affected)

Total execution time: 00:00:00.086

Country,StateProvince,TotalOrders
NULL,NULL,31465
Australia,NULL,6843
Australia,New South Wales,3009
Australia,Victoria,1710
Australia,Queensland,1461
Australia,South Australia,466
Australia,Tasmania,197
Canada,NULL,4067
Canada,British Columbia,3472
Canada,Ontario,339


<span style="font-size: 12.000000pt; font-family: 'TimesNewRomanPS'; font-weight: 700">Problem 3</span>

<span style="font-size: 12.000000pt; font-family: 'TimesNewRomanPSMT'">Produce a report showing the total number of units for each product sold by AdventureWorks that also includes the category of each product and each product's sales rank and quintile within the category. Make sure the result set is alphabetical by category and in the appropriate rank order.&nbsp;</span>

In [13]:
SELECT sod.ProductID
    , c.Name AS Category
    , SUM(sod.OrderQty) AS TotalSold
    , RANK() OVER (PARTITION BY c.name ORDER BY c.Name, SUM(sod.OrderQty))  AS Rank 
    , NTILE(5) OVER (PARTITION BY c.name ORDER BY SUM(sod.OrderQty)) AS Quintile
FROM Sales.SalesOrderDetail sod 
INNER JOIN Production.Product p
ON sod.ProductID = p.ProductID
INNER JOIN Production.ProductSubCategory sc
ON p.ProductSubcategoryID = sc.ProductSubcategoryID
INNER JOIN Production.ProductCategory c 
ON sc.ProductCategoryID = c.ProductCategoryID 
GROUP BY c.Name, sod.ProductID
ORDER BY c.Name, Rank

(266 rows affected)

Total execution time: 00:00:00.115

ProductID,Category,TotalSold,Rank,Quintile
879,Accessories,249,1,1
933,Accessories,858,2,1
928,Accessories,862,3,1
932,Accessories,926,4,1
934,Accessories,935,5,1
931,Accessories,1044,6,2
843,Accessories,1087,7,2
844,Accessories,1130,8,2
929,Accessories,1161,9,2
930,Accessories,1396,10,2
